In [ ]:
%%capture
!pip install streamlit streamlit-chat langchain transformers sentence-transformers faiss-cpu pypdf2 torch pyngrok
!pip install -q transformers einops accelerate
!pip install pytesseract
!apt-get install tesseract-ocr

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.5 MB/s eta 0:00:00


In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 15.6 MB/s eta 0:00:00


In [ ]:
# Cell 4: Mount Google Drive (if needed)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cell 2: Main Application Code
%%writefile app.py
import streamlit as st
import torch
import gc
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import HuggingFacePipeline
import tempfile
import logging
import time
from google.colab import drive
import shutil

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ResearchPaperChatbot:
    def __init__(self, model_name: str = "distilgpt2"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        st.info(f"Using device: {self.device}")
        self.setup_model(model_name)
        self.setup_embeddings()
        self.vector_store = None
        self.chain = None
        self.memory = None

    def setup_model(self, model_name: str):
      with st.spinner("Loading language model..."):
          try:
              print(f"Loading model: {model_name}")
              # Initialize tokenizer
              self.tokenizer = AutoTokenizer.from_pretrained(model_name)

              # Load model
              self.model = AutoModelForCausalLM.from_pretrained(
                  model_name,
                  device_map="auto",
                  torch_dtype=torch.float32,
                  low_cpu_mem_usage=True
              )

              print("Setting up pipeline...")
              # Setup pipeline
              self.pipe = pipeline(
                  "text-generation",
                  model=self.model,
                  tokenizer=self.tokenizer,
                  max_new_tokens=200,
                  do_sample=True,
                  temperature=0.7,
                  top_p=0.95,
                  return_full_text=False
              )

              print("Creating LLM...")
              # Create LLM
              self.llm = HuggingFacePipeline(pipeline=self.pipe)

              st.success("Model loaded successfully!")
              print("Model setup complete")

          except Exception as e:
              print(f"Model setup error: {e}")
              st.error(f"Error loading model: {str(e)}")
              raise

    def setup_embeddings(self):
        with st.spinner("Setting up embeddings..."):
            try:
                self.embeddings = HuggingFaceEmbeddings(
                    model_name="sentence-transformers/all-mpnet-base-v2",
                    model_kwargs={'device': self.device}
                )
                st.success("Embeddings setup complete!")
            except Exception as e:
                print(f"Error in setup_embeddings: {str(e)}")
                st.error(f"Error setting up embeddings: {str(e)}")
                raise

    def process_pdf(self, pdf_file):
        try:
            with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as tmp_file:
                tmp_file.write(pdf_file.getvalue())
                tmp_file_path = tmp_file.name

            loader = PyPDFLoader(tmp_file_path)
            documents = loader.load()

            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=500,
                chunk_overlap=50,
                length_function=len,
                separators=["\n\n", "\n", " ", ""]
            )

            texts = text_splitter.split_documents(documents)
            print(f"Split PDF into {len(texts)} chunks")

            os.unlink(tmp_file_path)
            return texts

        except Exception as e:
            print(f"Error processing PDF: {str(e)}")
            raise

    def initialize_chain(self, texts):
        with st.spinner("Creating vector store and initializing chain..."):
            try:
                print("Creating vector store...")
                self.vector_store = FAISS.from_documents(texts, self.embeddings)

                retriever = self.vector_store.as_retriever(
                    search_kwargs={
                        "k": 3,
                        "fetch_k": 5
                    }
                )

                self.memory = ConversationBufferMemory(
                    memory_key="chat_history",
                    return_messages=True,
                    output_key='answer'
                )

                self.chain = ConversationalRetrievalChain.from_llm(
                    llm=self.llm,
                    retriever=retriever,
                    memory=self.memory,
                    return_source_documents=True,
                    verbose=True
                )

                print("Chain initialized successfully")
                st.success("Document processed successfully!")

            except Exception as e:
                print(f"Error in initialize_chain: {str(e)}")
                st.error(f"Error processing document: {str(e)}")
                raise

    def generate_summary(self):
      try:
          print("Starting summary generation...")
          if not self.vector_store:
              return "Please upload a paper first."

          # Get document content
          docs = self.vector_store.similarity_search(
              "main points of this research paper",
              k=3
          )

          # Combine content
          content = " ".join([doc.page_content for doc in docs])
          print(f"Retrieved content length: {len(content)}")

          # Simple prompt
          input_text = f"Summarize this research paper: {content}"

          try:
              print("Generating summary...")
              # Use pipeline directly
              result = self.pipe(
                  input_text,
                  max_new_tokens=200,
                  do_sample=True,
                  temperature=0.7,
                  top_p=0.95,
                  num_return_sequences=1,
                  return_full_text=False
              )

              print("Generation completed")
              print(f"Result type: {type(result)}")
              print(f"Result content: {result}")

              # Extract the generated text
              if result and isinstance(result, list) and len(result) > 0:
                  if isinstance(result[0], dict) and 'generated_text' in result[0]:
                      return result[0]['generated_text'].strip()
                  else:
                      return str(result[0]).strip()
              else:
                  return "Could not generate summary."

          except Exception as gen_error:
              print(f"Generation error: {gen_error}")
              return f"Generation error: {str(gen_error)}"

      except Exception as e:
          print(f"Summary error: {e}")
          return f"Summary error: {str(e)}"

    def ask_question(self, question: str):
        try:
            if not self.chain:
                return "Please upload a paper first."

            response = self.chain({"question": question})
            return response['answer']

        except Exception as e:
            print(f"Error processing question: {str(e)}")
            return f"Error: {str(e)}"

    def format_summary(self, summary):
        sections = ["Objectives:", "Methodology:", "Key Findings:", "Conclusions:"]
        formatted = "Research Paper Summary\n\n"

        lines = summary.split('\n')
        current_section = ""

        for line in lines:
            line = line.strip()
            if any(section.lower() in line.lower() for section in sections):
                current_section = line
                formatted += f"\n{current_section}\n"
            elif line:
                formatted += f"{line}\n"

        return formatted


    def save_to_drive(self, file_path: str, drive_path: str):
        try:
            drive_full_path = f"/content/drive/My Drive/{drive_path}"
            os.makedirs(os.path.dirname(drive_full_path), exist_ok=True)
            shutil.copy2(file_path, drive_full_path)
            return True
        except Exception as e:
            st.error(f"Error saving to Drive: {str(e)}")
            return False

    def manage_memory(self):
      """Clean up memory and GPU cache"""
      try:
          # Clear CUDA cache if using GPU
          if torch.cuda.is_available():
              torch.cuda.empty_cache()

          # Clear conversation memory if exists
          if hasattr(self, 'memory') and self.memory is not None:
              self.memory.clear()

          # Perform garbage collection
          gc.collect()

      except Exception as e:
          print(f"Error in manage_memory: {str(e)}")

def initialize_session_state():
    if 'chatbot' not in st.session_state:
        st.session_state.chatbot = None
    if 'messages' not in st.session_state:
        st.session_state.messages = []
    if 'paper_processed' not in st.session_state:
        st.session_state.paper_processed = False

def main():
    st.title("Research Paper Analysis Chatbot (Colab Version)")

    initialize_session_state()

    with st.sidebar:
        st.header("Settings")

        model_options = {
            "GPT-2": "gpt2",
            "DistilGPT2": "distilgpt2",
            "BERT": "bert-base-uncased"
        }
        selected_model = st.selectbox(
            "Select Language Model",
            list(model_options.keys())
        )

        source = st.radio("Select source:", ["Upload File", "Google Drive"])

        if source == "Upload File":
            uploaded_file = st.file_uploader("Upload Research Paper (PDF)", type="pdf")
        else:
            drive_file = st.text_input("Enter path in Google Drive (e.g., 'papers/research.pdf')")
            if drive_file:
                full_path = f"/content/drive/My Drive/{drive_file}"
                if os.path.exists(full_path):
                    uploaded_file = open(full_path, 'rb')
                else:
                    st.error("File not found in Drive")
                    uploaded_file = None

        if uploaded_file and not st.session_state.paper_processed:
            try:
                if not st.session_state.chatbot:
                    st.session_state.chatbot = ResearchPaperChatbot(
                        model_name=model_options[selected_model]
                    )

                with st.spinner("Processing paper..."):
                    texts = st.session_state.chatbot.process_pdf(uploaded_file)
                    st.session_state.chatbot.initialize_chain(texts)
                    st.session_state.paper_processed = True

                st.success("Paper processed successfully!")

                with st.spinner("Generating summary..."):
                  print("\n=== Starting Summary Process ===")
                  if hasattr(st.session_state.chatbot, 'memory'):
                    st.session_state.chatbot.memory.clear()

                  print("Calling generate_summary...")
                  summary = st.session_state.chatbot.generate_summary()

                  print(f"\nSummary generation complete. Result length: {len(summary) if summary else 0}")
                  if summary and not isinstance(summary, str) or not summary.startswith("Error"):
                    print("Summary generated successfully")

                    st.session_state.messages.append({"role": "assistant","content": f"Summary of the paper:\n\n{summary}"})
                    print("Summary successfully added to messages")
                  else:
                      print(f"Summary generation failed: {summary}")
                      st.error(f"Could not generate summary: {summary}")

            except Exception as e:
              error_msg = f"Error in summary generation: {str(e)}"
              print(error_msg)
              st.error(error_msg)


    st.header("Chat Interface")

    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.write(message["content"])

    if st.session_state.paper_processed:
        user_question = st.chat_input("Ask a question about the paper")

        if user_question:
            st.session_state.messages.append({
                "role": "user",
                "content": user_question
            })

            with st.chat_message("user"):
                st.write(user_question)

            with st.chat_message("assistant"):
                with st.spinner("Thinking..."):
                    response = st.session_state.chatbot.ask_question(user_question)
                    st.write(response)

                    st.session_state.messages.append({
                        "role": "assistant",
                        "content": response
                    })

                    # Periodic memory cleanup
                    if len(st.session_state.messages) % 5 == 0:
                        st.session_state.chatbot.manage_memory()
    else:
        st.info("Please upload a research paper to start the conversation.")

    st.markdown("---")
    st.markdown("""
    ### How to use:
    1. Select a language model from the sidebar
    2. Choose file source (Upload or Google Drive)
    3. Upload or select your research paper
    4. Wait for the initial summary
    5. Ask questions about the paper

    ### Tips:
    - Be specific in your questions
    - You can ask about methods, results, conclusions, etc.
    - The bot remembers conversation context
    """)

if __name__ == "__main__":
    main()

Writing app.py


In [ ]:
# Cell 2: Setup ngrok authentication
from pyngrok import ngrok

In [ ]:
!ngrok authtoken 2riOJUDurYrJk5mxROf2umSvvMB_7itD6GvxQWtksw2VtvHLG


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import time
def run_app():
       !streamlit run app.py &>/content/logs.txt &
       !killall ngrok
       # Disconnect existing tunnels before connecting a new one
        #  !pkill -f "ngrok http"  # or

       # Wait for ngrok to shut down
       time.sleep(5)

       public_url = ngrok.connect(addr='8501')
       print(f"Public URL: {public_url}")

In [ ]:
run_app()

ngrok: no process found
Public URL: NgrokTunnel: "https://0048-34-143-249-121.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!killall ngrok

ngrok: no process found
